# About

This notebook converts Skills.xlsx into Skills.csv

In [110]:
import pandas as pd
import numpy as np

In [47]:
import json

# Read the SOC mapping JSON file
with open('../data/soc_mapping.json', 'r') as f:
    soc_mapping = json.load(f)

df = pd.DataFrame.from_dict(soc_mapping, orient='index')

# Reset index to make the SOC codes a column
df = df.reset_index()
df = df.rename(columns={'index': 'SOC Code'})
# First remove the hyphen, then take the appropriate number of digits
df['normalized_SOC_Code'] = df['SOC Code'].str.replace('-', '')
df['normalized_major_code'] = df['SOC Code'].str.replace('-', '').str[:2]
df['normalized_minor_code'] = df['SOC Code'].str.replace('-', '').str[:3]
df['normalized_broad_code'] = df['SOC Code'].str.replace('-', '').str[:4]

print("\nColumns in the DataFrame:")
print(df.columns.tolist())


Columns in the DataFrame:
['SOC Code', 'detailed_title', 'major_code', 'major_title', 'minor_code', 'minor_title', 'broad_code', 'broad_title', 'normalized_SOC_Code', 'normalized_major_code', 'normalized_minor_code', 'normalized_broad_code']


In [ ]:
# df.to_csv('../data/soc_mapping.csv', index=False)

In [74]:
skills_df = pd.read_excel('../data/ONET/Skills.xlsx')

In [75]:
skills_df.head()

,O*NET-SOC Code,Title,Element ID,Element Name,Scale ID,Scale Name,Data Value,N,Standard Error,Lower CI Bound,Upper CI Bound,Recommend Suppress,Not Relevant,Date,Domain Source
0,11-1011.00,Chief Executives,2.A.1.a,Reading Comprehension,IM,Importance,4.12,8,0.1250,3.8800,4.3700,N,NaN,08/2023,Analyst
1,11-1011.00,Chief Executives,2.A.1.a,Reading Comprehension,LV,Level,4.62,8,0.1830,4.2664,4.9836,N,N,08/2023,Analyst
2,11-1011.00,Chief Executives,2.A.1.b,Active Listening,IM,Importance,4.00,8,0.0000,4.0000,4.0000,N,NaN,08/2023,Analyst
3,11-1011.00,Chief Executives,2.A.1.b,Active Listening,LV,Level,4.75,8,0.1637,4.4292,5.0708,N,N,08/2023,Analyst
4,11-1011.00,Chief Executives,2.A.1.c,Writing,IM,Importance,4.12,8,0.1250,3.8800,4.3700,N,NaN,08/2023,Analyst


In [90]:
# print unique values of Element Name
print(f"unique count of Element Name: {len(skills_df['Element Name'].unique())}")
print(f"unique values of Element Name: {skills_df['Element Name'].unique()}")

print(f"unique count of O*NET-SOC Code: {len(skills_df['O*NET-SOC Code'].unique())}")
print(f"unique count of SOC code without .specification: {len(skills_df['O*NET-SOC Code'].unique())}")

unique count of Element Name: 35
unique values of Element Name: ['Reading Comprehension' 'Active Listening' 'Writing' 'Speaking'
 'Mathematics' 'Science' 'Critical Thinking' 'Active Learning'
 'Learning Strategies' 'Monitoring' 'Social Perceptiveness' 'Coordination'
 'Persuasion' 'Negotiation' 'Instructing' 'Service Orientation'
 'Complex Problem Solving' 'Operations Analysis' 'Technology Design'
 'Equipment Selection' 'Installation' 'Programming'
 'Operations Monitoring' 'Operation and Control' 'Equipment Maintenance'
 'Troubleshooting' 'Repairing' 'Quality Control Analysis'
 'Judgment and Decision Making' 'Systems Analysis' 'Systems Evaluation'
 'Time Management' 'Management of Financial Resources'
 'Management of Material Resources' 'Management of Personnel Resources']
unique count of O*NET-SOC Code: 879
unique count of SOC code without .specification: 879


In [25]:
# Update SOC_Code column based on the condition
skills_df['SOC_Code'] = skills_df['O*NET-SOC Code'].apply(lambda x: x.split('.')[0] if '.' in x else x)

In [76]:
# skills_df.to_csv('Skills.csv', index=False)

In [77]:
skills_importance_df = skills_df[skills_df['Scale Name'] == 'Importance']

In [78]:
skills_importance_df  = skills_importance_df.pivot(
    index=['O*NET-SOC Code', 'Title'],
    columns='Element Name',
    values='Data Value'
).reset_index()

In [79]:
skills_importance_df['normalized_major_code'] = skills_importance_df['O*NET-SOC Code'].str.replace('-', '').str[:2]
skills_importance_df['normalized_minor_code'] = skills_importance_df['O*NET-SOC Code'].str.replace('-', '').str[:3]


In [80]:
skills_importance_df

Element Name,O*NET-SOC Code,Title,Active Learning,Active Listening,Complex Problem Solving,Coordination,Critical Thinking,Equipment Maintenance,Equipment Selection,Installation,...,Social Perceptiveness,Speaking,Systems Analysis,Systems Evaluation,Technology Design,Time Management,Troubleshooting,Writing,normalized_major_code,normalized_minor_code
0,11-1011.00,Chief Executives,3.75,4.00,4.38,4.25,4.38,1.00,1.12,1.00,...,4.12,4.25,4.12,4.25,1.75,4.00,1.50,4.12,11,111
1,11-1011.03,Chief Sustainability Officers,3.75,4.00,4.00,3.75,4.12,1.00,1.12,1.00,...,3.88,4.00,3.88,3.88,1.88,3.38,1.00,4.12,11,111
2,11-1021.00,General and Operations Managers,3.62,4.00,3.62,3.88,3.88,1.00,1.00,1.00,...,3.75,4.00,3.12,3.12,1.50,3.62,1.75,3.50,11,111
3,11-2011.00,Advertising and Promotions Managers,3.25,4.12,3.50,3.50,4.00,1.00,1.12,1.00,...,4.00,4.00,3.12,3.12,1.75,3.50,1.00,3.75,11,112
4,11-2021.00,Marketing Managers,3.88,3.88,3.62,3.50,3.88,1.00,1.00,1.00,...,3.88,3.88,3.25,3.50,1.75,3.50,1.00,3.25,11,112
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
874,53-7071.00,Gas Compressor and Gas Pumping Station Operators,2.88,3.12,3.00,3.00,3.62,3.12,2.12,1.00,...,2.75,3.00,2.62,2.00,1.88,3.00,3.12,3.00,53,537
875,53-7072.00,"Pump Operators, Except Wellhead Pumpers",2.88,3.12,2.88,2.88,3.12,2.88,2.25,1.12,...,2.75,3.12,2.38,2.12,1.88,3.00,3.00,2.88,53,537
876,53-7073.00,Wellhead Pumpers,2.38,2.88,3.00,2.25,3.12,3.12,2.25,1.25,...,2.50,3.00,2.00,2.00,1.50,2.75,3.12,2.62,53,537
877,53-7081.00,Refuse and Recyclable Material Collectors,2.25,2.88,2.38,2.62,2.75,2.75,1.75,1.00,...,2.50,2.88,1.38,1.38,1.00,2.50,2.50,2.50,53,537


In [86]:
skills_importance_df.sort_values(by='Operations Analysis', ascending=False)[:30][['O*NET-SOC Code', 'Title', 'Operations Analysis']]

Element Name,O*NET-SOC Code,Title,Operations Analysis
123,17-1011.00,"Architects, Except Landscape and Naval",4.00
128,17-2011.00,Aerospace Engineers,3.88
152,17-2161.00,Nuclear Engineers,3.75
326,27-1027.00,Set and Exhibit Designers,3.75
130,17-2031.00,Bioengineers and Biomedical Engineers,3.75
117,15-2031.00,Operations Research Analysts,3.75
150,17-2141.02,Automotive Engineers,3.75
113,15-1299.08,Computer Systems Engineers/Architects,3.75
131,17-2041.00,Chemical Engineers,3.62
34,11-9111.00,Medical and Health Services Managers,3.62


In [81]:
skills_importance_df.to_csv('../data/skills/skills_importance.csv', index=False)

In [87]:
# Define your skill_categories mapping
skill_categories = {
    'basic_skills': [
        'Reading Comprehension', 'Active Listening', 'Writing', 'Speaking',
    ],
    'cognitive_skills': [
        'Critical Thinking', 'Active Learning', 'Learning Strategies',
        'Monitoring', 'Complex Problem Solving', 'Judgment and Decision Making', 'Operations Analysis'
    ],
    'social_skills': [
        'Social Perceptiveness', 'Coordination', 'Persuasion',
        'Negotiation', 'Instructing', 'Service Orientation'
    ],
    'operations_skills': [
        'Operation and Control', 'Operations Monitoring', 'Quality Control Analysis', 'Troubleshooting'
    ],
    'maintenance_skills': [
        'Equipment Selection', 'Installation', 'Equipment Maintenance', 'Repairing'
    ],
    'technical_skills': [
        'Technology Design', 'Programming', 'Mathematics', 'Science'
    ],
    'management_skills': [
        'Systems Analysis', 'Systems Evaluation', 'Time Management',
        'Management of Financial Resources', 'Management of Material Resources',
        'Management of Personnel Resources'
    ]
}

# Prepare a dict for the new DataFrame
new_data = {
    'O*NET-SOC Code': skills_importance_df['O*NET-SOC Code'],
    'Title': skills_importance_df['Title'],
}

# For each category, compute the mean of the mapped columns and add as a new column
for cat, skills in skill_categories.items():
    present_skills = [s for s in skills if s in skills_importance_df.columns]
    new_data[cat] = skills_importance_df[present_skills].mean(axis=1)

# Create the new DataFrame
skills_combined_df = pd.DataFrame(new_data)

# Display the first few rows to check
display(skills_combined_df.head())

,O*NET-SOC Code,Title,basic_skills,cognitive_skills,social_skills,operations_skills,maintenance_skills,technical_skills,management_skills
0,11-1011.00,Chief Executives,4.1225,3.928571,3.831667,1.8150,1.03,2.0925,4.145000
1,11-1011.03,Chief Sustainability Officers,4.0300,3.680000,3.521667,1.7200,1.03,2.1900,3.231667
2,11-1021.00,General and Operations Managers,3.8750,3.462857,3.500000,2.0650,1.00,1.7800,3.288333
3,11-2011.00,Advertising and Promotions Managers,3.9050,3.357143,3.333333,1.3100,1.03,2.0300,3.038333
4,11-2021.00,Marketing Managers,3.7225,3.625714,3.478333,1.4075,1.00,2.0325,3.188333


In [88]:
skills_combined_df.columns

Index(['O*NET-SOC Code', 'Title', 'basic_skills', 'cognitive_skills',
       'social_skills', 'operations_skills', 'maintenance_skills',
       'technical_skills', 'management_skills'],
      dtype='object')

In [108]:
import re
# ------------------------------------------------------------
# 1.  sector‑growth keywords for the WEF “discount” step
# ------------------------------------------------------------
GROWTH_SECTOR_PATTERNS = re.compile(
    r"(nurs|therap|counsel|teacher|educat|"      # care & education
    r"ai\b|ml\b|machine learning|data|cyber|"    # digital / AI / security
    r"engineer|developer|analyst|"               # generic digital titles
    r"renewable|solar|wind|green|sustain|"       # green transition
    r"project manager|operations manager)",      # leadership / project
    flags=re.I
)

# ------------------------------------------------------------
# 2.  helper to normalize each 1‑to‑5 skill bucket to 0‑1
# ------------------------------------------------------------
def _norm(series: pd.Series) -> pd.Series:
    return (series - 1.0) / 4.0

# ------------------------------------------------------------
# 3.  main scorer
# ------------------------------------------------------------
def add_automation_risk(df: pd.DataFrame) -> pd.DataFrame:
    """Append a 0‑100 'automation_risk_score' column to the skills dataframe.

    Expected numeric columns (1‑5 scale):
      basic_skills, cognitive_skills, social_skills,
      operations_skills, maintenance_skills,
      technical_skills, management_skills
    """
    # 1. normalise buckets
    bn  = _norm(df["basic_skills"])
    cn  = _norm(df["cognitive_skills"])
    sn  = _norm(df["social_skills"])
    on  = _norm(df["operations_skills"])
    mn  = _norm(df["maintenance_skills"])
    tn  = _norm(df["technical_skills"])
    man = _norm(df["management_skills"])

    # 2. composite indices
    routine_intensity = 0.5 * on + 0.5 * mn
    human_capital     = (
        0.40 * sn + 0.30 * man + 0.20 * cn + 0.10 * bn
    )   # ↑ social/management weight, ↓ basic
    tech_shield       = 0.30 * tn + 0.10 * cn

    # 3. raw risk (0‑1) with updated weights
    raw = (
        0.55 * routine_intensity
        + 0.30 * (1 - human_capital)
        + 0.15 * (1 - tech_shield)
    )

    # 4. WEF growth‑sector discount (‑30 %) – stricter threshold
    mask_growth = df["Title"].str.contains(GROWTH_SECTOR_PATTERNS, na=False) & (
        (sn >= 0.60) | (tn >= 0.60)   # need ≥3.4 in raw 1‑5 scale
    )
    raw = raw.mask(mask_growth, raw * 0.70)

    # 5. scale to 0‑100
    df = df.copy()
    df["automation_risk"] = (raw * 100).round(1)
    df["automation_risk_score"] = (
        100 / (1 + np.exp(-4 * (raw - 0.40)))
    ).round(1)


    return df



In [111]:
# ------------------------------------------------------------
# 4.  usage
# ------------------------------------------------------------
skills_combined_df = add_automation_risk(skills_combined_df)

/var/folders/0j/t51m6qr94pj38572cgsxl75h0000gp/T/ipykernel_58433/2844805351.py:55: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask_growth = df["Title"].str.contains(GROWTH_SECTOR_PATTERNS, na=False) & (


In [113]:
skills_combined_df.sort_values(by='automation_risk_score', ascending=False)[:20][['Title', 'automation_risk_score', 'automation_risk']]

,Title,automation_risk_score,automation_risk
722,Signal and Track Switch Repairers,76.0,68.9
705,Industrial Machinery Mechanics,75.5,68.2
655,Elevator and Escalator Installers and Repairers,75.2,67.8
680,"Electric Motor, Power Tool, and Related Repairers",74.5,66.8
706,"Maintenance Workers, Machinery",74.1,66.2
693,"Mobile Heavy Equipment Mechanics, Except Engines",73.9,66.0
687,Aircraft Mechanics and Service Technicians,73.9,66.1
692,Farm Equipment Mechanics and Service Technicians,73.6,65.6
695,Motorboat Mechanics and Service Technicians,73.2,65.2
697,Outdoor Power Equipment and Other Small Engine...,72.6,64.3


In [104]:
print(f'Mean: {skills_combined_df["automation_risk_score"].mean()}')
print(f'Median: {skills_combined_df["automation_risk_score"].median()}')
print(f'Std: {skills_combined_df["automation_risk_score"].std()}')


Mean: 41.77667804323094
Median: 39.4
Std: 11.42026791490296


In [114]:
skills_combined_df.to_csv('../data/skills/skills_based_risk.csv', index=False)